In [1]:
import numpy as np
from pyscf import gto, scf, df, ao2mo

In [2]:
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='STO-3G', charge = 1)
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='pc-3', charge = 1)
#mol = gto.M(atom='Be 0 0 0', basis='6-31G', charge = 0)
#mol = gto.M(atom='H 0 0 0; H 0 0 0.740848149', basis='STO-3G', charge = 0)
#mol = gto.M(atom="O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116", basis = "sto-3g")
mol = gto.M(atom="H 0.8668114 0.6014355  -0.000000000000; O 0.00000 -0.075791807925  0.00000; H -0.8668114 0.6014355  -0.000000000000", basis = "sto-3g")

In [3]:
#calculate nuclear repulsion energy
tot_nuc_energy=0
for i in range(0,len(mol.atom_coords())-1):
    for j in range(0,len(mol.atom_coords())):
        if j > i:
            print("i",i,"-- j",j)
            x=mol.atom_coord(i)-mol.atom_coord(j)
           # print("x",x)
            norm=np.linalg.norm(x)
           # print("norm",norm)
            nuc_energy=mol.atom_charge(i)*mol.atom_charge(j)/norm
            print("nuc_energy",nuc_energy)     
            tot_nuc_energy+=nuc_energy
print("tot_nuc_energy",tot_nuc_energy)

i 0 -- j 1
nuc_energy 3.8485633950000477
i 0 -- j 2
nuc_energy 0.3052435690855012
i 1 -- j 2
nuc_energy 3.8485633950000477
tot_nuc_energy 8.002370359085596


In [4]:
#get integrals and calculate core Hamiltonian

overlap_integrals = mol.intor('int1e_ovlp_sph')
#print("overlap_integrals")
#print(overlap_integrals)
kinetic_energy_integrals = mol.intor('int1e_kin_sph')
#print("kinetic_energy_integrals")
#print(kinetic_energy_integrals)
nuclear_attraction_integrals = mol.intor('int1e_nuc_sph')
#print("nuclear_attraction_integrals")
#print(nuclear_attraction_integrals)
two_electron_integrals = mol.intor('int2e_sph')
#print("two_electron_integrals")
#print(two_electron_integrals)
H_core = kinetic_energy_integrals + nuclear_attraction_integrals


In [5]:
#get integrals and calculate core Hamiltonian

#overlap_integrals = mol.intor('int1e_ovlp_spinor')
#print("overlap_integrals")
#print(overlap_integrals)
#kinetic_energy_integrals = mol.intor('int1e_kin_spinor')
#print("kinetic_energy_integrals")
#print(kinetic_energy_integrals)
#nuclear_attraction_integrals = mol.intor('int1e_nuc_spinor')
#print("nuclear_attraction_integrals")
#print(nuclear_attraction_integrals)
#two_electron_integrals = mol.intor('int2e_spinor')
#print("two_electron_integrals")
#print(two_electron_integrals)
#H_core = kinetic_energy_integrals + nuclear_attraction_integrals


In [6]:
#diagonalize overlap matrix
overlap_diag_procedure = np.linalg.eigh(overlap_integrals)
#get U and S^-0.5
U = overlap_diag_procedure[1]
s_inverse_sqrt = np.diag(1/overlap_diag_procedure[0])**0.5
#get transformation matrix X
X = np.matmul(U,np.matmul(s_inverse_sqrt,np.matrix.transpose(U)))

In [7]:
#set guess for P matrix, default core hamiltonian
P = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

In [8]:
#start iterations
counter = 0
maxiter = 100
energy_difference=10000
energy_last_iter=10000
energy=0

for counter in range(0,maxiter):
    G = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    G[i,j]+=P[k,l]*(two_electron_integrals[i,j,k,l]-0.5*two_electron_integrals[i,l,k,j])

    F = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    F = H_core + G
#    print(F)
    
    if counter >= 1:
        energy = 0
        for i in range(0,overlap_integrals.shape[0]):
            for j in range(0,overlap_integrals.shape[0]):
                energy += 0.5*(P[i,j]*(H_core[i,j]+F[i,j]))
        energy_difference = abs(energy-energy_last_iter)
        print('iteration {:<9}'.format(counter),"energy = {:<25}".format(energy), "difference = {:<23}".format(energy_difference))
        energy_last_iter = energy
    if energy_difference < 10**-12:
        print("energy converged")
        SCF_final_energy = energy + tot_nuc_energy
        print("final SCF energy including nuclear repulsion:",SCF_final_energy)
        break
    counter+=1    

    F_prime = np.matmul(np.matrix.transpose(X),np.matmul(F,X))
    F_diag_procedure = np.linalg.eigh(F_prime)
    
    C = np.matmul(X,F_diag_procedure[1])
    
    P_new = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    
    for i in range(0,overlap_integrals.shape[0]):
        for j in range(0,overlap_integrals.shape[1]):    
            for a in range(0,mol.nelectron//2):
                P_new[i,j]+=2*C[i,a]*C[j,a]

    P = P_new

iteration 1         energy = -81.28816666315855        difference = 10081.288166663158     
iteration 2         energy = -82.83049643078779        difference = 1.5423297676292407     
iteration 3         energy = -82.93785857072511        difference = 0.10736213993732235    
iteration 4         energy = -82.94384827368513        difference = 0.005989702960022214   
iteration 5         energy = -82.94434252432465        difference = 0.0004942506395195778  
iteration 6         energy = -82.94442658921184        difference = 8.406488719003846e-05  
iteration 7         energy = -82.94444494684353        difference = 1.8357631688559195e-05 
iteration 8         energy = -82.94444917532057        difference = 4.228477038736855e-06  
iteration 9         energy = -82.94445015781449        difference = 9.82493915557825e-07   
iteration 10        energy = -82.94445038647702        difference = 2.286625289116273e-07  
iteration 11        energy = -82.94445043971035        difference = 5.3233335961

In [9]:
C

array([[-4.59374706e-03,  1.44039575e-01,  4.52997579e-01,
        -3.29471253e-01,  2.18805489e-18, -7.09849625e-01,
         7.32461027e-01],
       [ 9.94434589e-01, -2.39158838e-01, -8.40391986e-16,
        -9.36833035e-02,  1.48337510e-17, -1.11639958e-01,
         1.66010698e-16],
       [ 2.40970479e-02,  8.85735485e-01,  3.87368306e-15,
         4.79586254e-01, -7.17893056e-17,  6.69579495e-01,
        -1.38787878e-16],
       [-3.44528968e-16, -1.92779242e-15,  6.07285003e-01,
        -6.67159197e-15, -6.71773630e-17,  3.41918106e-15,
        -9.19234337e-01],
       [ 3.16155239e-03,  8.58962946e-02, -7.19293346e-15,
        -7.47431248e-01,  1.39713033e-16,  7.38488850e-01,
         1.96084654e-15],
       [-3.07730003e-19,  2.48816633e-18,  1.00735196e-16,
         1.69762769e-16,  1.00000000e+00, -1.76573902e-17,
        -6.52977057e-18],
       [-4.59374706e-03,  1.44039575e-01, -4.52997579e-01,
        -3.29471253e-01,  8.38880496e-17, -7.09849625e-01,
        -7.3246102

In [10]:
np.set_printoptions(precision=3)
np.dot(C.T,np.dot(F,C))

array([[-2.026e+01, -4.166e-09, -2.646e-15, -3.028e-09, -5.318e-18,
        -1.832e-09, -2.486e-15],
       [-4.166e-09, -1.210e+00, -9.885e-16,  2.650e-08,  2.276e-18,
         1.883e-08,  1.173e-16],
       [-2.926e-15, -7.674e-16, -5.480e-01,  2.309e-15,  1.701e-17,
        -1.631e-15, -1.143e-07],
       [-3.028e-09,  2.650e-08,  1.986e-15, -4.365e-01,  1.064e-17,
         1.493e-07,  9.868e-16],
       [-5.318e-18,  2.276e-18,  1.701e-17,  1.064e-17, -3.876e-01,
         6.905e-18,  2.880e-18],
       [-1.832e-09,  1.883e-08, -1.787e-15,  1.493e-07,  6.905e-18,
         4.776e-01, -6.615e-17],
       [-2.887e-15,  5.551e-16, -1.143e-07,  1.388e-15,  2.880e-18,
        -2.220e-16,  5.881e-01]])

In [203]:
#doitthedumbway
#C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))
#C_ao_mo[:,:]=C[:,:]

#integrals_pqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))

#for p in range(0,integrals_pqrs.shape[0]):
 ##   #print(p)
   # for q in range(0,integrals_pqrs.shape[1]):
    #    for r in range(0,integrals_pqrs.shape[2]):
     #       for s in range(0,integrals_pqrs.shape[3]):
      #          for i in range(0,two_electron_integrals.shape[0]):
       #             for j in range(0,two_electron_integrals.shape[1]):
        #                for k in range(0,two_electron_integrals.shape[2]):
         #                   for l in range(0,two_electron_integrals.shape[3]):
          #                      integrals_pqrs[p,q,r,s]+=C_ao_mo[i,p]*C_ao_mo[j,q]*C_ao_mo[k,r]*C_ao_mo[l,s]*two_electron_integrals[i,j,k,l]

#print(integrals_pqrs)

In [204]:
#doitthesmartway
C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))
C_ao_mo[:,:]=C[:,:]

integrals_ijks = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_ijrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_iqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_pqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))

for s in range(0,integrals_ijks.shape[0]):
    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    integrals_ijks[i,j,k,s]+=C_ao_mo[l,s]*two_electron_integrals[i,j,k,l]
    
for s in range(0,integrals_ijrs.shape[0]):
    for r in range(0,integrals_ijrs.shape[1]):
        for i in range(0,two_electron_integrals.shape[0]):
            for j in range(0,two_electron_integrals.shape[1]):
                for k in range(0,two_electron_integrals.shape[2]):
                    integrals_ijrs[i,j,r,s]+=C_ao_mo[k,r]*integrals_ijks[i,j,k,s]

for s in range(0,integrals_iqrs.shape[0]):
    for r in range(0,integrals_iqrs.shape[1]):
        for q in range(0,integrals_iqrs.shape[2]):
            for i in range(0,two_electron_integrals.shape[0]):
                for j in range(0,two_electron_integrals.shape[1]):
                    integrals_iqrs[i,q,r,s]+=C_ao_mo[j,q]*integrals_ijrs[i,j,r,s]

for s in range(0,integrals_pqrs.shape[0]):
    for r in range(0,integrals_pqrs.shape[1]):
        for q in range(0,integrals_pqrs.shape[2]):
            for p in range(0,integrals_pqrs.shape[3]):
                for i in range(0,two_electron_integrals.shape[0]):
                    integrals_pqrs[p,q,r,s]+=C_ao_mo[i,p]*integrals_iqrs[i,q,r,s]
                    
print(integrals_pqrs)


[[[[ 4.747e+00 -4.283e-01 -1.272e-15 ...  2.686e-17 -1.998e-01
     9.828e-16]
   [-4.283e-01  1.019e+00  5.847e-16 ... -2.005e-17  2.689e-01
     1.213e-15]
   [-1.259e-15  5.912e-16  7.630e-01 ...  3.307e-17 -4.808e-16
    -3.701e-01]
   ...
   [ 2.686e-17 -2.005e-17  3.307e-17 ...  1.115e+00  3.869e-17
     3.548e-17]
   [-1.998e-01  2.689e-01 -4.742e-16 ...  3.869e-17  7.820e-01
    -1.338e-15]
   [ 9.831e-16  1.290e-15 -3.701e-01 ...  3.548e-17 -1.379e-15
     8.397e-01]]

  [[-4.283e-01  6.109e-02  1.385e-16 ... -3.393e-18  3.036e-02
    -9.212e-18]
   [ 6.109e-02 -1.514e-02 -1.290e-16 ...  1.334e-18 -6.137e-03
     6.959e-17]
   [ 1.339e-16 -1.365e-16 -4.671e-03 ... -7.039e-19 -1.442e-17
     7.016e-03]
   ...
   [-3.393e-18  1.334e-18 -7.039e-19 ... -1.211e-02 -4.652e-19
    -6.859e-19]
   [ 3.036e-02 -6.137e-03 -1.946e-17 ... -4.652e-19 -7.415e-03
     2.795e-17]
   [-1.303e-17  5.264e-17  7.016e-03 ... -6.859e-19  4.312e-17
    -8.921e-03]]

  [[-1.269e-15  1.369e-16  1.059e-

In [181]:
#np.max(integrals_pqrs-np.load("VeeMO.npy"))

In [193]:
#def integraltransformation(I, C):
#    # N^5 scaling
#    K = np.einsum('pi,ijkl->pjkl', C, I)
#    K = np.einsum('qj,pjkl->pqkl', C, K)
#    K = np.einsum('rk,pqkl->pqrl', C, K)
#    K = np.einsum('sl,pqrl->pqrs', C, K)
#    return K

In [194]:
#C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))
#C_ao_mo[:,:]=C[:,:]

#for i in range(0,C_ao_mo.shape[0]):
#    for p in range(0,C_ao_mo.shape[1]):
#        C_ao_mo[i,p]=C[i,p]

#mf = scf.RHF(mol).run()
#mo_ints = ao2mo.kernel(mol, mf.mo_coeff)
#mo_integrals = ao2mo.restore(1, mo_ints, mol.nao_nr())
 
#mo_integrals = integraltransformation(two_electron_integrals,C_ao_mo)
#print(mo_integrals)

In [195]:
#mo_integrals_spin = np.zeros((len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals))*2)
#for p in range(0,len(mo_integrals_spin)):
#    for r in range(0,len(mo_integrals_spin)):
#        print("pr:", p,r)
#        if (p%2==r%2):
#            for q in range(0,len(mo_integrals_spin)):
#                for s in range(0,len(mo_integrals_spin)):
#                    if (q%2==s%2):
#                        print("qs:", q,s)
#                        mo_integrals_spin[p,q,r,s]=mo_integrals[p//2,q//2,r//2,s//2]
#mo_integrals_spin

In [222]:
mo_integrals = integrals_pqrs
mo_integrals_spin = np.zeros((len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2))
for p in range(1,len(mo_integrals_spin)+1):
    for r in range(1,len(mo_integrals_spin)+1):
        if (p%2==r%2):
            #print("pr:", p,r)
            for q in range(1,len(mo_integrals_spin)+1):
                for s in range(1,len(mo_integrals_spin)+1):
                    if (q%2==s%2):
                        #print("qs:", q,s)
                        mo_integrals_spin[p-1,q-1,r-1,s-1]=mo_integrals[(p+1)//2-1,(r+1)//2-1,(q+1)//2-1,(s+1)//2-1]
mo_integrals_spin

array([[[[ 4.747e+00,  0.000e+00, -4.283e-01, ...,  0.000e+00,
           9.828e-16,  0.000e+00],
         [ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,
           0.000e+00,  0.000e+00],
         [-4.283e-01,  0.000e+00,  6.109e-02, ...,  0.000e+00,
          -9.212e-18,  0.000e+00],
         ...,
         [ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,
           0.000e+00,  0.000e+00],
         [ 9.862e-16,  0.000e+00, -1.256e-17, ...,  0.000e+00,
           1.952e-02,  0.000e+00],
         [ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,
           0.000e+00,  0.000e+00]],

        [[ 0.000e+00,  4.747e+00,  0.000e+00, ..., -1.998e-01,
           0.000e+00,  9.828e-16],
         [ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,
           0.000e+00,  0.000e+00],
         [ 0.000e+00, -4.283e-01,  0.000e+00, ...,  3.036e-02,
           0.000e+00, -9.212e-18],
         ...,
         [ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,
           0.000e+00,  0.

In [223]:
#def Transform2eSPIN(VeeMO):
#    VeeMOspin = np.zeros((len(VeeMO)*2,len(VeeMO)*2,len(VeeMO)*2,len(VeeMO)*2))
#    for p in range(1,len(VeeMO)*2+1):
#        for r in range(1,len(VeeMO)*2+1):
#            if (p%2 == r%2):
#                for q in range(1,len(VeeMO)*2+1):
#                    for s in range(1,len(VeeMO)*2+1):
#                        if (q%2 == s%2):
#                            VeeMOspin[p-1,q-1,r-1,s-1] = VeeMO[(p+1)//2-1,(r+1)//2-1,(q+1)//2-1,(s+1)//2-1]
#    return VeeMOspin
#
#mo_integrals_spin=Transform2eSPIN(integrals_pqrs)

In [224]:
#np.load("VeeMOspin.npy")

In [225]:
B = np.zeros((mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron),mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron)))

arindex = -1
for a in range(0,mol.nelectron):
    for r in range(mol.nelectron,overlap_integrals.shape[0]*2):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron):
            for s in range(mol.nelectron,overlap_integrals.shape[0]*2):
                bsindex += 1
                B[arindex,bsindex]+=mo_integrals_spin[r,s,a,b]-mo_integrals_spin[r,s,b,a]
                #print(arindex,bsindex)
               # lol[a,b] = 1
               # lol[r,s] = 2
                #print(r,s,a,b,mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a],mo_integrals[r,s,a,b],mo_integrals[r,s,b,a])
#assert np.allclose(B, np.zeros(B.shape))
print(np.max(np.abs(B)))
print(B)
print(B-np.loadtxt("B.txt"))
np.max(B-np.loadtxt("B.txt"))

0.1594488785288076
[[ 0.000e+00  0.000e+00  0.000e+00 ... -3.111e-19  0.000e+00 -5.704e-19]
 [ 0.000e+00  0.000e+00  0.000e+00 ...  0.000e+00 -2.857e-19  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 ...  2.857e-19  0.000e+00 -1.780e-18]
 ...
 [-3.111e-19  0.000e+00  2.857e-19 ...  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -2.857e-19  0.000e+00 ...  0.000e+00  0.000e+00  0.000e+00]
 [-5.704e-19  0.000e+00 -1.780e-18 ...  0.000e+00  0.000e+00  0.000e+00]]
[[ 0.000e+00  0.000e+00  0.000e+00 ... -3.108e-19  0.000e+00 -5.726e-19]
 [ 0.000e+00  0.000e+00  0.000e+00 ...  0.000e+00 -2.875e-19  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 ...  2.875e-19  0.000e+00 -1.782e-18]
 ...
 [-3.108e-19  0.000e+00  2.875e-19 ...  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -2.875e-19  0.000e+00 ...  0.000e+00  0.000e+00  0.000e+00]
 [-5.726e-19  0.000e+00 -1.782e-18 ...  0.000e+00  0.000e+00  0.000e+00]]


6.044273515026788e-08

In [226]:
#print(np.loadtxt("B.txt"))
#print(np.max(np.abs(np.loadtxt("B.txt"))))

In [227]:
eigenenergies=np.zeros((len(F_diag_procedure[0])*2))
for i in range(0,len(eigenenergies)):
    eigenenergies[i]+= np.dot(C.T,np.dot(F,C))[i//2][i//2]
print(eigenenergies)

A = np.zeros((mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron),mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron)))

arindex = -1
for a in range(0,mol.nelectron):
    for r in range(mol.nelectron,overlap_integrals.shape[0]*2):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron):
            for s in range(mol.nelectron,overlap_integrals.shape[0]*2):
                bsindex += 1
                A[arindex,bsindex]+=mo_integrals_spin[r,b,a,s]-mo_integrals_spin[r,b,s,a]
                if a==b and r==s:
                    A[arindex,bsindex] += eigenenergies[r] - eigenenergies[a]
                    #print(eigenenergies[r])
                    #print(eigenenergies[a])
                    
print(A)
#print(A-np.loadtxt("A.txt"))

[-20.263 -20.263  -1.21   -1.21   -0.548  -0.548  -0.437  -0.437  -0.388
  -0.388   0.478   0.478   0.588   0.588]
[[ 1.998e+01  0.000e+00  1.277e-15 ... -3.111e-19  0.000e+00 -5.704e-19]
 [ 0.000e+00  1.996e+01  0.000e+00 ...  0.000e+00  0.000e+00  0.000e+00]
 [ 1.307e-15  0.000e+00  2.003e+01 ...  2.857e-19  0.000e+00 -1.780e-18]
 ...
 [-3.111e-19  0.000e+00  2.857e-19 ...  3.219e-01  0.000e+00  7.594e-16]
 [ 0.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  3.660e-01  0.000e+00]
 [-5.704e-19  0.000e+00 -1.780e-18 ...  7.737e-16  0.000e+00  3.910e-01]]


In [228]:
#np.max(A-np.loadtxt("A.txt"))

In [229]:
mat = np.matmul((A+B),(A-B)) 
print(mat)

[[ 3.994e+02  0.000e+00  5.111e-14 ... -3.942e-19  0.000e+00 -7.531e-19]
 [ 0.000e+00  3.983e+02  0.000e+00 ...  0.000e+00  5.496e-18  0.000e+00]
 [ 5.228e-14  0.000e+00  4.012e+02 ...  1.282e-20  0.000e+00 -1.739e-18]
 ...
 [-1.283e-17  0.000e+00  1.126e-17 ...  1.036e-01  0.000e+00  5.412e-16]
 [ 0.000e+00 -5.496e-18  0.000e+00 ...  0.000e+00  1.333e-01  0.000e+00]
 [-2.309e-17  0.000e+00 -7.200e-17 ...  5.514e-16  0.000e+00  1.529e-01]]


In [230]:
Exc = np.real(np.sort(np.sqrt(np.linalg.eigvals(mat))))
print("hartree                  ","eV")
print("------------------------------------------")
for i in range(0,len(Exc)): 
    print(Exc[i],"      ", Exc[i]*27.2116)

hartree                   eV
------------------------------------------
0.2851640173193956        7.759769173688466
0.28516401731939583        7.759769173688472
0.285164017319397        7.759769173688504
0.2997439226838388        8.156511726503549
0.29974392268387595        8.15651172650456
0.2997439226839157        8.156511726505641
0.352627203026287        9.595550397870111
0.3526272030262897        9.595550397870184
0.3526272030263117        9.595550397870783
0.354778578106846        9.65409275601225
0.3651317393826693        9.935818839385444
0.3651317393826695        9.93581883938545
0.36513173938266963        9.935818839385453
0.41531790071414076        11.301464587072912
0.5001014802853665        13.60856144093328
0.5106615063294392        13.895916645634168
0.5106615063294392        13.895916645634168
0.5106615063294441        13.895916645634301
0.5460723998484831        14.859503715716983
0.5460723998484878        14.85950371571711
0.5460723998485179        14.85950371571793
0

In [231]:
print(Exc)

[ 0.285  0.285  0.285  0.3    0.3    0.3    0.353  0.353  0.353  0.355
  0.365  0.365  0.365  0.415  0.5    0.511  0.511  0.511  0.546  0.546
  0.546  0.551  0.65   0.873  1.104  1.104  1.104  1.196  1.196  1.196
  1.283  1.324 19.959 19.959 19.959 20.011 20.011 20.011 20.011 20.05 ]


In [160]:
#mat_diag_procedure = np.linalg.eigh(mat)
#print("hartree", mat_diag_procedure[0]**(0.5))
#print("eV", mat_diag_procedure[0]**(0.5)*27.2116)

In [36]:
#for i in range(0,len(F)):
#    print(np.matmul(np.transpose(C),(np.matmul(F,C)))[i,i])

In [291]:
#np.shape(np.load("VeeMOspin.npy"))
#np.load("VeeMO.npy")

In [292]:
#np.shape(mo_integrals_spin)

In [293]:
#(mo_integrals-np.load("VeeMO.npy"))[:5,:5,:5,:5]

In [154]:
def RPA(occ, F, C, VeeMOspin):
    # Make the spin MO fock matrix
    Fspin = np.zeros((len(F)*2,len(F)*2))
    Cspin = np.zeros((len(F)*2,len(F)*2))
    for p in range(1,len(F)*2+1):
        for q in range(1,len(F)*2+1):
            Fspin[p-1,q-1] = F[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
            Cspin[p-1,q-1] = C[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
    FMOspin = np.dot(np.transpose(Cspin),np.dot(Fspin,Cspin))
    #print(FMOspin)


    #Construct hamiltonian
    A = np.zeros((occ*(len(Fspin)-occ),occ*(len(Fspin)-occ)))
    B = np.zeros((occ*(len(Fspin)-occ),occ*(len(Fspin)-occ)))
    jbidx = -1
    for j in range(0, occ):
        for b in range(occ, len(Fspin)):
            jbidx += 1
            iaidx = -1
            for i in range(0, occ):
                for a in range(occ, len(Fspin)):
                    iaidx += 1
                    A[iaidx,jbidx] = VeeMOspin[a,j,i,b] - VeeMOspin[a,j,b,i]
                    B[iaidx,jbidx] = VeeMOspin[a,b,i,j] - VeeMOspin[a,b,j,i]
                    if i == j:
                        A[iaidx,jbidx] += FMOspin[a,b]
                        #if (np.abs(FMOspin[a,b]) > 10E-5):
                           # print(FMOspin[a,b])
                    if a == b:
                        A[iaidx,jbidx] -= FMOspin[i,j]
                        #if (np.abs(FMOspin[i,j]) > 10E-5):
                          #  print(FMOspin[i,j])
    
    C = np.dot(A+B,A-B)
    #print(np.max(C-mat))
    Exc = np.sort(np.sqrt(np.linalg.eigvals(C)))
    
    return Exc


In [153]:
RPA(10,F,C,mo_integrals_spin)

8.82776337585156e-08


array([ 0.285+0.000e+00j,  0.285+0.000e+00j,  0.285+0.000e+00j,
        0.3  +0.000e+00j,  0.3  +0.000e+00j,  0.3  +0.000e+00j,
        0.353+0.000e+00j,  0.353-1.264e-15j,  0.353+1.264e-15j,
        0.355+0.000e+00j,  0.365+0.000e+00j,  0.365+0.000e+00j,
        0.365+0.000e+00j,  0.415+0.000e+00j,  0.5  +0.000e+00j,
        0.511-3.286e-15j,  0.511+3.286e-15j,  0.511+0.000e+00j,
        0.546+0.000e+00j,  0.546+0.000e+00j,  0.546+0.000e+00j,
        0.551+0.000e+00j,  0.65 +0.000e+00j,  0.873+0.000e+00j,
        1.104+0.000e+00j,  1.104+0.000e+00j,  1.104+0.000e+00j,
        1.196+0.000e+00j,  1.196-9.667e-15j,  1.196+9.667e-15j,
        1.283+0.000e+00j,  1.324+0.000e+00j, 19.959+0.000e+00j,
       19.959+0.000e+00j, 19.959+0.000e+00j, 20.011+0.000e+00j,
       20.011+0.000e+00j, 20.011+0.000e+00j, 20.011+0.000e+00j,
       20.05 +0.000e+00j])

In [309]:
np.max(np.load("Vee.npy") - two_electron_integrals)

1.5835730859947716e-07

In [190]:
integrals_ijks = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_ijrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_iqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_pqrs_2 = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
#C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))


#for i in range(0,C_ao_mo.shape[0]):
#    for p in range(0,C_ao_mo.shape[1]):
#        C_ao_mo[i,p]=C[i,p]


for s in range(0,integrals_ijks.shape[0]):
    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    integrals_ijks[i,j,k,s]+=C_ao_mo[l,s]*two_electron_integrals[i,j,k,l]
    
for s in range(0,integrals_ijrs.shape[0]):
    for r in range(0,integrals_ijrs.shape[1]):
        for i in range(0,two_electron_integrals.shape[0]):
            for j in range(0,two_electron_integrals.shape[1]):
                for k in range(0,two_electron_integrals.shape[2]):
                    integrals_ijrs[i,j,r,s]+=C_ao_mo[k,r]*integrals_ijks[i,j,k,s]

for s in range(0,integrals_iqrs.shape[0]):
    for r in range(0,integrals_iqrs.shape[1]):
        for q in range(0,integrals_iqrs.shape[2]):
            for i in range(0,two_electron_integrals.shape[0]):
                for j in range(0,two_electron_integrals.shape[1]):
                    integrals_iqrs[i,q,r,s]+=C_ao_mo[j,q]*integrals_ijrs[i,j,r,s]

for s in range(0,integrals_pqrs.shape[0]):
    for r in range(0,integrals_pqrs.shape[1]):
        for q in range(0,integrals_pqrs.shape[2]):
            for p in range(0,integrals_pqrs.shape[3]):
                for i in range(0,two_electron_integrals.shape[0]):
                    integrals_pqrs_2[p,q,r,s]+=C_ao_mo[i,p]*integrals_iqrs[i,q,r,s]
                    
    
#for s in range(0,integrals_pqrs.shape[0]):
 #   for r in range(0,integrals_pqrs.shape[1]):
  #      for q in range(0,integrals_pqrs.shape[2]):
   #         for p in range(0,integrals_pqrs.shape[3]):             
print(np.max(integrals_pqrs_2-integrals_pqrs))



1.1546319456101628e-14


In [66]:
B = np.zeros((mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0]))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,B.shape[1]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,B.shape[3]):
                B[a,r,b,s]+=integrals_pqrs[r,s,a,b]-integrals_pqrs[r,s,b,a]
B

array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

      

In [67]:
A = np.zeros((mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0]))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,B.shape[1]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,B.shape[3]):
                A[a,r,b,s]+=mo_integrals[r,b,a,s]-mo_integrals[r,b,s,a]
                if a==b and r==s:
                    A[a,r,b,s]+= (F_diag_procedure[0][r] - F_diag_procedure[0][a])

#for i in range(0,A.shape[0]):
#    A[i,i,i,i]=A[i,i,i,i]+

In [ ]:
B = np.zeros((mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2),mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2)))

#mf = scf.RHF(mol).run()
#mo_ints = ao2mo.kernel(mol, mf.mo_coeff)
#mo_ints = ao2mo.restore(1, mo_ints, mol.nao_nr())
#mo_integrals = mo_ints

arindex = -1
for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,overlap_integrals.shape[0]):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,overlap_integrals.shape[0]):
                bsindex += 1
                B[arindex,bsindex]+=2*mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a]
               # lol[a,b] = 1
               # lol[r,s] = 2
                #print(r,s,a,b,mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a],mo_integrals[r,s,a,b],mo_integrals[r,s,b,a])
#assert np.allclose(B, np.zeros(B.shape))
print(B)

In [ ]:
A = np.zeros((mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2),mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2)))

arindex = -1
for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,overlap_integrals.shape[0]):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,overlap_integrals.shape[0]):
                bsindex += 1
                A[arindex,bsindex]+=2*mo_integrals[r,b,a,s]-mo_integrals[r,b,s,a]
                if a==b:
                    A[arindex,bsindex] += F_diag_procedure[0][r]
                if r==s:
                    A[arindex,bsindex] += - F_diag_procedure[0][a]
                    
print(A)